In [ ]:
# 구글 드라이브 Mount
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
path = '/content/drive/MyDrive/AKI/mimic'

## 주제

분당 서울대병원에서 하는 mimic-4 데이터셋을 이용한 급성 신손상 예측 모델 개발이 목표

## 급성 신장 손상이란?

연구하고 있는 게
- NAP 데이터를 이용해서 aki prediction 해서 검사에 어떤 특정 수치가 변화하면 바로 진단이 가능하게끔
- Serial data 로 냅 데이터가 이런 추세로 가다가 다른 데이터를 참고했을 때 수치가 이렇게 떨어지겠지 -> 조기 진단 (진단 기준에 들어가기만 하면 됨)
- 계속 검사를 받다가 특정 시점에 변화가 나타나 진단

질병 코드(ICD-10)?으로 나와 있음


급성 신장 손상(AKI)은 혈청 크레아티닌(sCr) 증가 또는 소변량 감소로 인식되는 신장 기능의 갑작스러운 감소를 특징으로 합니다. 주로 다음 두 가지 지표를 통해 진단됩니다:
1. 혈청 크레아티닌(Serum Creatinine, sCr) 증가:

- 크레아티닌은 근육에서 생성되는 노폐물로, 신장을 통해 배출됩니다. 신장이 제대로 기능하지 않을 경우 혈액 내 크레아티닌 수치가 상승하게 됩니다.
- AKI는 일반적으로 다음과 같은 혈청 크레아티닌 수치 변화를 기준으로 진단됩니다:
  - 기준치에 비해 혈청 크레아티닌이 48시간 내에 0.3 mg/dL 이상 증가한 경우.
  - 기준치에 비해 혈청 크레아티닌이 7일 이내에 1.5배 이상 증가한 경우.

2. 소변량 감소:

- 신장이 적절히 기능하지 않을 경우, 소변을 충분히 배출하지 못하게 됩니다. 이는 신장의 여과 능력이 저하되었음을 의미합니다.
- AKI는 다음과 같은 소변량 변화를 기준으로 진단됩니다:
  - 6시간 이상 동안 시간당 체중 kg당 0.5mL 미만의 소변량이 지속될 경우.

이 두 가지 진단 기준 중 하나라도 만족하면 AKI로 진단될 수 있습니다.

### 급성 신손상 기준에 대하여

- 생체 신호 측정과 실험실 지표는 모두 환자가 ICU에 입원한 후 처음 24시간 동안 기록된 초기 값을 기반으로 한다는 점
- 환자가 ICU에 입원한 후 7일 이내의 AKI 발생률에 초점
- Kidney Disease: Improving Global Outcomes(KDIGO) 가이드라인 참조할 것
  - 48시간 이내에 혈청 크레아티닌(SCr) 수치가 ≥26.5μmol/L(0.3mg/dl) 증가, 기준 값에 비해 SCr 값이 ≥50% 증가(1.5배 증가), 또는 6시간 이상 소변량이 0.5mL/kg/h 미만입니다.중요한 점은 이 진단 프레임워크에서 중요한 매개변수인 기준 SCr이 바로 전주에 관찰된 가장 낮은 SCr 값으로 정의된다는 것


**단계별 분류 (KDIGO 기준)**

AKI는 심각도에 따라 1단계(Stage 1)에서 3단계(Stage 3)로 구분됩니다. 이는 주로 혈청 크레아티닌 수치와 소변량을 기준으로 나누어지며, 단계가 높을수록 신장 기능 저하가 심각하다는 것을 의미합니다.

- 1단계 (Stage 1):
  - 혈청 크레아티닌 수치가 기준치보다 1.5배에서 1.9배 증가하거나, 48시간 내에 0.3 mg/dL 이상 증가.
  - 6시간 동안 체중 kg당 시간당 0.5mL 미만의 소변량.
- 2단계 (Stage 2):
  - 혈청 크레아티닌 수치가 기준치보다 2배에서 2.9배 증가.
  - 12시간 동안 체중 kg당 시간당 0.5mL 미만의 소변량.
- 3단계 (Stage 3):
  - 혈청 크레아티닌 수치가 기준치보다 3배 이상 증가하거나, 4.0 mg/dL 이상 증가.
  - 24시간 동안 체중 kg당 시간당 0.3mL 미만의 소변량 또는 무뇨(oliguria: 하루에 100mL 이하의 소변량) 상태가 12시간 이상 지속될 때.
  - 신대체요법(Renal Replacement Therapy, RRT)을 시작해야 할 경우.

**원인**

AKI는 다양한 원인에 의해 발생할 수 있으며, 크게 세 가지로 분류됩니다:

- 신전성 (Pre-renal) AKI:
  - 신장 자체는 정상이나, 신장으로 가는 혈류가 감소하여 발생합니다.
  - 예: 탈수, 출혈, 심부전, 저혈압.
- 신성 (Intrinsic) AKI:
  - 신장 자체의 구조적 손상이 원인입니다.
  - 예: 급성 세뇨관 괴사(acute tubular necrosis, ATN), 급성 사구체신염(acute glomerulonephritis), 약물 독성.
- 신후성 (Post-renal) AKI:
  - 신장에서 소변을 배출하는 경로의 폐쇄로 인해 발생합니다.
  - 예: 요로 폐쇄, 전립선 비대, 결석.

## 환자 기준

급성 췌장염으로 진단받은 환자는 국제질병분류 9차 개정판(ICD-9, 코드 577.0)과 10차 개정판(ICD10, 코드 K85%)을 사용하여 데이터베이스에서 확인했습니다. 신중한 제외 기준에는 18세 미만의 환자, ICU에 24시간 미만 체류한 환자, 유지 신장 대체 요법(투석 또는 이식)을 받는 환자, 신장 질환의 기록이 있는 개인이 포함되었습니다. 여러 번 ICU에 입원한 환자의 경우, 데이터 검색은 최초 입원에 대해서만 수행하여 방법론적 일관성을 보장했습니다.

## 개발하려는 모델

- 입원기간(7일 이내) 동안 급성 신손상 발생 여부를 예측하는 구조로 환자의 데이터(나이, 성별, 진단명, 투약정보, 검사결과 등)가 입력되면 급성 신손상이 발생할지 여부를 알려주는 방식이다.
- 입원 후 24시간, 48시간, 72시간 시점의 혈청 크레아티닌 수치를 예측하는 구조
https://www.whosaeng.com/127096

## 데이터셋 탐구

mimic 데이터셋 파일 구조
- ICU 데이터 (icu 폴더): 중환자실에서 상세한 환자 모니터링 정보
  - 환자의 체내 상태를 지속적으로 모니터링한 기록, 처치 기록, 약물 투여 기록 등이 포함된다. 주로 매우 짧은 시간 간격으로 수집된 시계열 데이터로 구성되어 있어, 중환자실에서 발생한 급성 상태 변화를 예측하기 위해 사용된다.
- 병원 데이터 (hosp 폴더): 병원 전체에서 관리되는 환자의 일반적인 기록
  - 입원 기록, 진단 정보, 처방 정보, 실험실 결과 등이 포함된다. 병원 전반에서 발생하는 환자의 상태를 포괄적으로 관리하기 위한 데이터다.

## ICU 데이터셋 구조 및 주요 컬럼 설명

1. `caregiver.csv.gz`
- **설명**: 중환자실에서 환자에게 처치를 수행한 의료진의 정보입니다. 각 의료진의 ID, 직위, 역할 등이 포함됩니다.
- **주요 컬럼**:
  - `caregiver_id`: 의료진의 고유 ID

---

2. `chartevents.csv.gz`
- **설명**: 환자의 바이탈 사인, 검사 결과, 약물 투여 등의 시계열 데이터를 포함하는 가장 큰 테이블입니다. 중환자실에서 주기적으로 측정된 환자의 상태 정보를 포함합니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `stay_id`: 중환자실 체류 ID
  - `caregiver_id`: 의료진 ID
  - `charttime`: 데이터가 기록된 시간
  - `storetime`: 데이터가 저장된 시간
  - `itemid`: 항목의 고유 ID
  - `value`: 측정된 값
  - `valuenum`: 측정된 수치 값
  - `valueuom`: 단위
  - `warning`: 경고 플래그

---

3. `d_items.csv.gz`
- **설명**: `itemid`와 같은 데이터 항목의 메타데이터를 제공하는 테이블입니다. 각 항목의 이름과 설명이 포함되어 있어, `inputevents`, `outputevents` 등의 다른 테이블에서 사용된 `itemid`의 의미를 알 수 있습니다.
- **주요 컬럼**:
  - `itemid`: 항목의 고유 ID
  - `label`: 항목의 라벨 (예: "Heart Rate", "Blood Pressure")
  - `abbreviation`: 약어
  - `linksto`: 항목이 속하는 이벤트 테이블
  - `category`: 항목의 카테고리
  - `unitname`: 단위 (예: bpm, mmHg)
  - `param_type`: 데이터 유형 (예: Numeric, Text)
  - `lownormalvalue`: 정상 하한 값
  - `highnormalvalue`: 정상 상한 값

---

4. `datetimeevents.csv.gz`
- **설명**: 중환자실 내에서 시간 관련 이벤트를 기록한 데이터입니다. 환자의 검사 결과, 바이탈 사인 측정 등과 관련된 시간 이벤트가 포함될 수 있습니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `stay_id`: 중환자실 체류 ID
  - `caregiver_id`: 의료진 ID
  - `charttime`: 이벤트가 기록된 시간
  - `storetime`: 이벤트가 데이터베이스에 저장된 시간
  - `itemid`: 시간 이벤트 항목의 고유 ID
  - `value`: 시간 값
  - `valueuom`: 단위
  - `warning`: 경고 플래그

---

5. `icustays.csv.gz`
- **설명**: 중환자실 입실 및 퇴실 정보입니다. 각 환자가 중환자실에 입실한 시간, 퇴실한 시간, 입원 이유 등이 포함됩니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `stay_id`: 중환자실 체류 ID
  - `first_careunit`: 처음 배정된 병동
  - `last_careunit`: 마지막 배정된 병동
  - `intime`: 중환자실 입실 시간
  - `outtime`: 중환자실 퇴실 시간
  - `los`: 중환자실 체류 기간 (일 단위)

---

6. `ingredientevents.csv.gz`
- **설명**: 복합 약물이나 수액 등 여러 성분이 포함된 처치 이벤트의 개별 성분 정보를 기록한 데이터입니다. 예를 들어, 특정 약물 성분의 투입량 등을 기록합니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `stay_id`: 중환자실 체류 ID
  - `caregiver_id`: 의료진 ID
  - `starttime`: 성분 투입 시작 시간
  - `endtime`: 성분 투입 종료 시간
  - `storetime`: 데이터가 저장된 시간
  - `itemid`: 성분 항목의 고유 ID
  - `amount`: 투입된 성분의 양
  - `amountuom`: 단위
  - `rate`: 투입 속도
  - `rateuom`: 속도 단위
  - `orderid`: 주문 ID
  - `linkorderid`: 연관 주문 ID
  - `statusdescription`: 상태 설명
  - `originalamount`: 원래 투입량
  - `originalrate`: 원래 투입 속도

---

7. `inputevents.csv.gz`
- **설명**: 환자에게 투여된 약물이나 영양제 등의 투입 관련 이벤트를 기록한 데이터입니다. 투입된 물질의 종류, 양, 투입 시간 등이 포함됩니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `stay_id`: 중환자실 체류 ID
  - `caregiver_id`: 의료진 ID
  - `starttime`: 투입 시작 시간
  - `endtime`: 투입 종료 시간
  - `storetime`: 데이터가 저장된 시간
  - `itemid`: 투입 항목의 고유 ID
  - `amount`: 투입된 양
  - `amountuom`: 단위
  - `rate`: 투입 속도
  - `rateuom`: 속도 단위
  - `orderid`: 주문 ID
  - `linkorderid`: 연관 주문 ID
  - `ordercategoryname`: 주문 카테고리 이름
  - `secondaryordercategoryname`: 2차 주문 카테고리 이름
  - `ordercomponenttypedescription`: 주문 구성 요소 설명
  - `ordercategorydescription`: 주문 카테고리 설명
  - `patientweight`: 환자 체중
  - `totalamount`: 총 투입량
  - `totalamountuom`: 총 투입량 단위
  - `isopenbag`: 개봉 여부
  - `continueinnextdept`: 다음 부서에서 계속 투입 여부
  - `statusdescription`: 상태 설명
  - `originalamount`: 원래 투입량
  - `originalrate`: 원래 투입 속도

---

8. `outputevents.csv.gz`
- **설명**: 환자의 체외로 배출된 물질(소변, 배설물, 배액 등)과 관련된 이벤트를 기록한 데이터입니다. 배출된 물질의 양과 시간 등을 포함합니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `stay_id`: 중환자실 체류 ID
  - `caregiver_id`: 의료진 ID
  - `charttime`: 배출 이벤트가 기록된 시간
  - `storetime`: 데이터가 저장된 시간
  - `itemid`: 배출 이벤트 항목의 고유 ID
  - `value`: 배출된 물질의 양
  - `valueuom`: 단위

---

9. `procedureevents.csv.gz`
- **설명**: 중환자실에서 이루어진 의료 절차나 수술 관련 이벤트를 기록한 데이터입니다. 여기에는 수술명, 시술 날짜와 시간, 시술 결과 등이 포함될 수 있습니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `stay_id`: 중환자실 체류 ID
  - `caregiver_id`: 의료진 ID
  - `starttime`: 절차 시작 시간
  - `endtime`: 절차 종료 시간
  - `storetime`: 데이터가 저장된 시간
  - `itemid`: 절차 항목의 고유 ID
  - `value`: 절차와 관련된 추가 정보
  - `valueuom`: 단위
  - `location`: 절차가 수행된 장소
  - `locationcategory`: 장소 카테고리
  - `orderid`: 주문 ID
  - `linkorderid`: 연관 주문 ID
  - `ordercategoryname`: 주문 카테고리 이름
  - `ordercategorydescription`: 주문 카테고리 설명
  - `patientweight`: 환자 체중
  - `isopenbag`: 개봉 여부
  - `continueinnextdept`: 다음 부서에서 계속 수행 여부
  - `statusdescription`: 상태 설명
  - `originalamount`: 원래 투입량
  - `originalrate`: 원래 투입 속도


In [ ]:
import os
import pandas as pd

# ICU 폴더만 탐색
for root, dirs, files in os.walk(path):
    # root에 icu 폴더가 포함된 경우만 탐색
    if '3.0/icu' in root:
        for file in files:
            if file.endswith('.csv.gz'):
                file_path = os.path.join(root, file)
                try:
                    # 파일 읽기 (처음 5줄만 읽어서 데이터 구조 확인)
                    df = pd.read_csv(file_path, nrows=5, compression='gzip')
                    print(f"파일명: {file_path}")
                    print(f"컬럼명: {df.columns.tolist()}")
                    print(f"데이터 샘플:\n{df.head()}\n")
                except Exception as e:
                    print(f"파일 '{file_path}'를 읽는 도중 에러 발생: {e}")

파일명: /content/drive/MyDrive/AKI/mimic/3.0/icu/chartevents.csv.gz
컬럼명: ['subject_id', 'hadm_id', 'stay_id', 'caregiver_id', 'charttime', 'storetime', 'itemid', 'value', 'valuenum', 'valueuom', 'warning']
데이터 샘플:
   subject_id   hadm_id   stay_id  caregiver_id            charttime  \
0    10000032  29079034  39553978         18704  2180-07-23 12:36:00   
1    10000032  29079034  39553978         18704  2180-07-23 12:36:00   
2    10000032  29079034  39553978         18704  2180-07-23 12:36:00   
3    10000032  29079034  39553978         18704  2180-07-23 14:00:00   
4    10000032  29079034  39553978         18704  2180-07-23 14:00:00   

             storetime  itemid              value  valuenum valueuom  warning  
0  2180-07-23 14:45:00  226512               39.4      39.4       kg        0  
1  2180-07-23 14:45:00  226707                 60      60.0     Inch        0  
2  2180-07-23 14:45:00  226730                152     152.0       cm        0  
3  2180-07-23 14:18:00  220048  SR (

### hosp 데이터

### 병원 데이터 (hosp 폴더)

### 1. `admissions.csv.gz`
- **설명**: 환자의 입원 정보가 포함된 테이블로, 입원 시각, 퇴원 시각, 입원 방식, 입원 이유 등의 정보가 기록되어 있습니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `admittime`: 입원 시각
  - `dischtime`: 퇴원 시각
  - `deathtime`: 사망 시각 (해당하는 경우)
  - `admission_type`: 입원 유형 (예: 응급, 계획)
  - `diagnosis`: 입원 시 진단

### 2. `d_hcpcs.csv.gz`
- **설명**: HCPCS(HCFA Common Procedure Coding System) 코드의 메타데이터를 포함하는 테이블입니다. 각 의료 절차의 HCPCS 코드와 그 설명이 포함되어 있습니다.
- **주요 컬럼**:
  - `code`: HCPCS 코드
  - `category`: 카테고리
  - `long_description`: 코드에 대한 긴 설명
  - `short_description`: 코드에 대한 짧은 설명

### 3. `d_icd_diagnoses.csv.gz`
- **설명**: ICD 진단 코드에 대한 메타데이터를 제공하는 테이블입니다. 각 진단 코드의 설명이 포함되어 있습니다.
- **주요 컬럼**:
  - `icd_code`: ICD 진단 코드
  - `icd_version`: ICD 버전 (예: 9, 10)
  - `long_title`: 진단 코드의 긴 설명

### 4. `d_icd_procedures.csv.gz`
- **설명**: ICD 절차 코드에 대한 메타데이터를 제공하는 테이블로, 각 절차 코드의 설명이 포함되어 있습니다.
- **주요 컬럼**:
  - `icd_code`: ICD 절차 코드
  - `icd_version`: ICD 버전 (예: 9, 10)
  - `long_title`: 절차 코드의 긴 설명

### 5. `d_labitems.csv.gz`
- **설명**: 실험실 검사 항목에 대한 메타데이터를 제공하는 테이블로, 각 항목의 ID와 설명이 포함되어 있습니다.
- **주요 컬럼**:
  - `itemid`: 실험실 검사 항목 ID
  - `label`: 검사 항목 이름 (예: "Creatinine", "Hemoglobin")
  - `fluid`: 검사 대상 유체 (예: 혈액, 소변)
  - `category`: 검사 카테고리 (예: 혈액 검사, 소변 검사)

### 6. `diagnoses_icd.csv.gz`
- **설명**: 입원 기간 동안 환자에게 내려진 ICD 진단 코드 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `seq_num`: 진단 순서 번호
  - `icd_code`: 진단 코드
  - `icd_version`: ICD 버전 (예: 9, 10)

### 7. `drgcodes.csv.gz`
- **설명**: 입원 기간 동안 환자에게 할당된 DRG(Diagnosis-Related Group) 코드 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `drg_type`: DRG 유형 (예: APR, HCFA)
  - `drg_code`: DRG 코드
  - `description`: DRG 코드 설명
  - `drg_severity`: DRG 중증도 (1~4)
  - `drg_mortality`: DRG 사망률 (1~4)

### 8. `emar.csv.gz`
- **설명**: 전자 투약 관리(Electronic Medication Administration Record, EMAR) 시스템을 통해 기록된 투약 이벤트 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `emar_id`: EMAR 이벤트 ID
  - `emar_seq`: EMAR 이벤트 순서 번호
  - `poe_id`: 처방 이벤트 ID
  - `pharmacy_id`: 약국 ID
  - `charttime`: 투약 기록 시간
  - `medication`: 투약된 약물 이름
  - `event_txt`: 투약 이벤트 텍스트 (예: "Administered", "Flushed")
  - `scheduletime`: 예정된 투약 시간
  - `storetime`: 투약 기록이 저장된 시간

### 9. `emar_detail.csv.gz`
- **설명**: `emar.csv.gz`에 있는 투약 이벤트의 상세 정보를 제공하는 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `emar_id`: EMAR 이벤트 ID
  - `emar_seq`: EMAR 이벤트 순서 번호
  - `parent_field_ordinal`: 상위 필드 순서 번호
  - `administration_type`: 투약 유형
  - `pharmacy_id`: 약국 ID
  - `dose_due`: 예정된 투약량
  - `dose_due_unit`: 예정된 투약량 단위
  - `dose_given`: 실제 투약량
  - `dose_given_unit`: 실제 투약량 단위

### 10. `hcpcsevents.csv.gz`
- **설명**: HCPCS 코드와 관련된 이벤트 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `chartdate`: 이벤트 기록 날짜
  - `hcpcs_cd`: HCPCS 코드
  - `seq_num`: 순서 번호
  - `short_description`: HCPCS 코드 설명

### 11. `labevents.csv.gz`
- **설명**: 실험실 검사 결과 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `labevent_id`: 실험실 검사 이벤트 ID
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `specimen_id`: 검체 ID
  - `itemid`: 검사 항목 ID
  - `order_provider_id`: 검사 주문 제공자 ID
  - `charttime`: 검사 기록 시간
  - `storetime`: 검사 결과가 저장된 시간
  - `value`: 검사 결과 값
  - `valuenum`: 검사 결과 수치 값
  - `valueuom`: 검사 결과 단위 (예: mg/dL)
  - `ref_range_lower`: 기준 하한값
  - `ref_range_upper`: 기준 상한값
  - `flag`: 비정상 여부 플래그

### 12. `microbiologyevents.csv.gz`
- **설명**: 미생물 검사 결과 및 배양 정보가 포함된 테이블입니다.
- **주요 컬럼**:
  - `microevent_id`: 미생물 검사 이벤트 ID
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `micro_specimen_id`: 미생물 검체 ID
  - `order_provider_id`: 검사 주문 제공자 ID
  - `chartdate`: 검사 기록 날짜
  - `charttime`: 검사 기록 시간
  - `spec_itemid`: 검사 항목 ID
  - `spec_type_desc`: 검사 유형 설명 (예: 혈액, 소변)
  - `test_itemid`: 검사 항목 ID
  - `test_name`: 검사 이름 (예: "Culture", "Sensitivity")
  - `org_itemid`: 배양된 미생물 항목 ID
  - `org_name`: 배양된 미생물 이름
  - `interpretation`: 검사 해석 결과 (예: Positive, Negative)

### 13. `omr.csv.gz`
- **설명**: OMR(Outpatient Medical Record) 정보를 포함하는 테이블로, 외래 기록 정보가 포함되어 있습니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `chartdate`: 기록된 날짜
  - `seq_num`: 순서 번호
  - `result_name`: 검사 결과 이름 (예: "Blood Pressure", "Weight")
  - `result_value`: 검사 결과 값

### 14. `patients.csv.gz`
- **설명**: 환자의 인구통계학적 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `gender`: 성별 (예: 남성, 여성)
  - `anchor_age`: 기준 나이
  - `anchor_year`: 기준 연도
  - `anchor_year_group`: 기준 연도 그룹
  - `dod`: 사망 날짜 (해당하는 경우)

### 15. `pharmacy.csv.gz`
- **설명**: 환자에게 처방된 약물 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `pharmacy_id`: 약국 ID
  - `poe_id`: 처방 이벤트 ID
  - `medication`: 처방된 약물 이름
  - `proc_type`: 처방 유형 (예: Unit Dose)
  - `status`: 처방 상태 (예: 활성, 비활성)
  - `entertime`: 처방 입력 시간
  - `starttime`: 약물 시작 시간
  - `stoptime`: 약물 중지 시간

### 16. `poe.csv.gz`
- **설명**: 처방 항목에 대한 이벤트 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `poe_id`: 처방 이벤트 ID
  - `poe_seq`: 처방 이벤트 순서 번호
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `ordertime`: 주문 시간
  - `order_type`: 주문 유형 (예: 약물, 검사)
  - `order_subtype`: 주문 하위 유형
  - `order_provider_id`: 주문 제공자 ID
  - `order_status`: 주문 상태 (예: 활성, 비활성)

### 17. `poe_detail.csv.gz`
- **설명**: 처방 항목의 상세 정보를 포함한 테이블입니다.
- **주요 컬럼**:
  - `poe_id`: 처방 이벤트 ID
  - `poe_seq`: 처방 이벤트 순서 번호
  - `subject_id`: 환자 ID
  - `field_name`: 필드 이름 (예: "Code status", "Admit category")
  - `field_value`: 필드 값 (예: "Full code", "Inpatient")

### 18. `prescriptions.csv.gz`
- **설명**: 환자에게 처방된 약물의 상세 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `pharmacy_id`: 약국 ID
  - `poe_id`: 처방 이벤트 ID
  - `poe_seq`: 처방 이벤트 순서 번호
  - `order_provider_id`: 처방 제공자 ID
  - `starttime`: 약물 시작 시간
  - `stoptime`: 약물 중지 시간
  - `drug_type`: 약물 유형 (예: "MAIN")
  - `drug`: 처방된 약물 이름
  - `dose_val_rx`: 처방된 약물의 양
  - `dose_unit_rx`: 단위 (예: mg)

### 19. `procedures_icd.csv.gz`
- **설명**: ICD 절차 코드 정보를 포함하는 테이블입니다. 각 절차와 관련된 환자의 정보가 포함되어 있습니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `seq_num`: 절차 순서 번호
  - `chartdate`: 절차 기록 날짜
  - `icd_code`: 절차 코드
  - `icd_version`: ICD 버전 (예: 9, 10)

### 20. `provider.csv.gz`
- **설명**: 환자에게 진료를 제공한 의료 제공자(의사, 간호사 등)의 정보가 포함된 테이블입니다.
- **주요 컬럼**:
  - `provider_id`: 제공자 ID

### 21. `services.csv.gz`
- **설명**: 환자에게 제공된 서비스 정보를 포함하는 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `transfertime`: 서비스 제공 시간
  - `prev_service`: 이전에 제공된 서비스
  - `curr_service`: 현재 제공된 서비스

### 22. `transfers.csv.gz`
- **설명**: 병원 내에서 환자의 병실 이동 정보를 기록한 테이블입니다.
- **주요 컬럼**:
  - `subject_id`: 환자 ID
  - `hadm_id`: 입원 ID
  - `transfer_id`: 이동 이벤트 ID
  - `eventtype`: 이벤트 유형 (예: 입실, 퇴실)
  - `careunit`: 병동 이름
  - `intime`: 병동 입실 시간
  - `outtime`: 병동 퇴실 시간



In [ ]:
# hosp 폴더만 탐색
for root, dirs, files in os.walk(path):
    # root에 hosp 폴더가 포함된 경우만 탐색
    if '3.0/hosp' in root:
        for file in files:
            if file.endswith('.csv.gz'):
                file_path = os.path.join(root, file)
                try:
                    # 파일 읽기 (처음 5줄만 읽어서 데이터 구조 확인)
                    df = pd.read_csv(file_path, nrows=5, compression='gzip')
                    print(f"파일명: {file_path}")
                    print(f"컬럼명: {df.columns.tolist()}")
                    print(f"데이터 샘플:\n{df.head()}\n")
                except Exception as e:
                    print(f"파일 '{file_path}'를 읽는 도중 에러 발생: {e}")

파일명: /content/drive/MyDrive/AKI/mimic/3.0/hosp/drgcodes.csv.gz
컬럼명: ['subject_id', 'hadm_id', 'drg_type', 'drg_code', 'description', 'drg_severity', 'drg_mortality']
데이터 샘플:
   subject_id   hadm_id drg_type  drg_code  \
0    10000032  22595853      APR       283   
1    10000032  22595853     HCFA       442   
2    10000032  22841357      APR       279   
3    10000032  22841357     HCFA       442   
4    10000032  25742920      APR       283   

                                         description  drg_severity  \
0                       OTHER DISORDERS OF THE LIVER           2.0   
1  DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...           NaN   
2  HEPATIC COMA AND OTHER MAJOR ACUTE LIVER DISOR...           3.0   
3  DISORDERS OF LIVER EXCEPT MALIG,CIRR,ALC HEPA ...           NaN   
4                       OTHER DISORDERS OF THE LIVER           3.0   

   drg_mortality  
0            2.0  
1            NaN  
2            2.0  
3            NaN  
4            2.0  

파일명: /content